In [1]:
# Required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt


In [66]:
ventas_proveedor1.head()

,Sku,Date,Store_Id,Units,Price,Cadena
0,4220015,2017-01-01 SO Diaria Piezas,18,6.0,15.6,proveedor_1
1,4220015,2017-01-01 SO Diaria Piezas,24,0.0,0.0,proveedor_1
2,4220015,2017-01-01 SO Diaria Piezas,39,0.0,0.0,proveedor_1
3,4220015,2017-01-01 SO Diaria Piezas,44,0.0,0.0,proveedor_1
4,4220015,2017-01-01 SO Diaria Piezas,46,0.0,0.0,proveedor_1


In [69]:
ventas_proveedor2.head()

,Sku,Date,Store_Id,Pieces,Price,Cadena
0,4240039,2016/01/01,1000,23.0,$16.00,proveedor_2
1,4240039,2016/01/01,1001,116.0,$16.00,proveedor_2
2,4240039,2016/01/01,1002,146.0,$16.00,proveedor_2
3,4240039,2016/01/01,1003,17.0,$16.80,proveedor_2
4,4240039,2016/01/01,1004,5.0,$16.70,proveedor_2


In [65]:
# Carga de datos

clientes = pd.read_csv('maestro_clientes_prueba.csv', sep = ';',low_memory=False)
productos = pd.read_csv('maestro_productos_prueba.csv', sep = ',',low_memory=False)
ventas_proveedor1 = pd.read_csv('sellout_proveedor1.csv', sep = ',',low_memory=False)
ventas_proveedor2 = pd.read_csv('sellout_proveedor2.csv', sep = ',',low_memory=False)

##
clientes.columns = ['Store_Id','Nombre_tienda','Cadena','Formato','Municipio','Estado','Division','Distribuidor','Estructura','Grado']
ventas_proveedor1['Cadena'] = 'proveedor_1'
ventas_proveedor2['Cadena'] = 'proveedor_2'
ventas_df = pd.concat([ventas_proveedor1,ventas_proveedor2])

print("dim clientes: " + str(clientes.shape))
print(clientes.head(2) , '\n')

print("dim productos: " + str(productos.shape))
print(productos.head(2) , '\n')

print("dim ventas: " + str(ventas_df.shape))
print(ventas_df.head(2) , '\n')

dim clientes: (2452, 10)
   Store_Id              Nombre_tienda       Cadena Formato  \
0         2  Bloody Sea Green BookShop  proveedor_1    Shop   
1         4  Funny Sky Blue GipsyTrade  proveedor_1    Shop   

            Municipio   Estado Division    Distribuidor      Estructura Grado  
0  Bizarre White Shop  Coalmar  SURESTE  Mogulval Corp.  DISTRIBUIDORES    VI  
1  Bizarre White Shop  Coalmar  SURESTE  Mogulval Corp.  DISTRIBUIDORES    VI   

dim productos: (1242, 4)
     Sku      Sku_Description   Sku_Category Sku_Subcategory
0  11058       Frosted Mango      Sugar-free     Cereal bars
1  29207  Nut Cheerios Mango   Super-flavour      Cereal box 

dim ventas: (6068394, 7)
       Sku                         Date  Store_Id  Units Price       Cadena  \
0  4220015  2017-01-01 SO Diaria Piezas        18    6.0  15.6  proveedor_1   
1  4220015  2017-01-01 SO Diaria Piezas        24    0.0   0.0  proveedor_1   

   Pieces  
0     NaN  
1     NaN   



In [64]:
ventas[(ventas['Cadena']=='proveedor_1') & (ventas['Units'].isna())]

,Sku,Date,Store_Id,Units,Price,Cadena,Pieces,Sku_Description,Sku_Category,Sku_Subcategory,Nombre_tienda,Formato,Municipio,Estado,Division,Distribuidor,Estructura,Grado


In [52]:
ventas.shape

(6073832, 18)

In [62]:
# Unir dataset e productos con cada proveedor
clientes['Cadena'] = clientes['Cadena'].str.strip()

ventas = ventas_df.merge(productos, on="Sku", how="left").merge(clientes, on=["Store_Id","Cadena"], how="left")

# Check for missing values
print(ventas.isnull().sum())

Sku                      0
Date                     0
Store_Id                 0
Units              4862352
Price                    0
Cadena                   0
Pieces             1211480
Sku_Description        465
Sku_Category           465
Sku_Subcategory        465
Nombre_tienda        34100
Formato              34100
Municipio            34100
Estado               34100
Division             34100
Distribuidor         34100
Estructura           34100
Grado                34100
dtype: int64


In [ ]:
# Merge sales data with product and store info for better analysis
sales_data = sales_df.merge(products_df, on="Sku", how="left").merge(stores_df, on="Store_Id", how="left")

# Check for missing values
print(sales_data.isnull().sum())

# Filling missing values if necessary (e.g., fill missing prices or units with the median)
sales_data['Price'] = sales_data['Price'].fillna(sales_data['Price'].median())
sales_data['Units'] = sales_data['Units'].fillna(sales_data['Units'].median())

# Convert the 'Date' column to datetime
sales_data['Date'] = pd.to_datetime(sales_data['Date'])

# Sort data by date for time series analysis
sales_data = sales_data.sort_values('Date')


(4862029, 5)


,Sku,Date,Store_Id,Units,Price
0,4220015,2017-01-01 SO Diaria Piezas,18,6.0,15.6
1,4220015,2017-01-01 SO Diaria Piezas,24,0.0,0.0
2,4220015,2017-01-01 SO Diaria Piezas,39,0.0,0.0
3,4220015,2017-01-01 SO Diaria Piezas,44,0.0,0.0
4,4220015,2017-01-01 SO Diaria Piezas,46,0.0,0.0


In [70]:
## Step 1: Data Exploration and Preparation

In [ ]:
# Required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Load datasets (assuming CSV format)
stores_df = pd.read_csv("stores.csv")
products_df = pd.read_csv("products.csv")
sales_df = pd.read_csv("sales.csv")

# Merge sales data with product and store info for better analysis
sales_data = sales_df.merge(products_df, on="Sku", how="left").merge(stores_df, on="Store_Id", how="left")

# Check for missing values
print(sales_data.isnull().sum())

# Filling missing values if necessary (e.g., fill missing prices or units with the median)
sales_data['Price'] = sales_data['Price'].fillna(sales_data['Price'].median())
sales_data['Units'] = sales_data['Units'].fillna(sales_data['Units'].median())

# Convert the 'Date' column to datetime
sales_data['Date'] = pd.to_datetime(sales_data['Date'])

# Sort data by date for time series analysis
sales_data = sales_data.sort_values('Date')


In [ ]:
## Step 2: Feature Engineering

In [ ]:
# Extract useful features from date (day of the week, month, etc.)
sales_data['Day_of_Week'] = sales_data['Date'].dt.dayofweek
sales_data['Month'] = sales_data['Date'].dt.month

# Create lag features (previous day’s sales for time series modeling)
sales_data['Lag_1'] = sales_data['Units'].shift(1)
sales_data['Lag_7'] = sales_data['Units'].shift(7)

# Remove NA values created by lags
sales_data = sales_data.dropna()


In [ ]:
## Step 3: Demand Prediction (Time Series Forecasting)

In [ ]:
# Aggregate sales per day (for all SKUs or per SKU)
daily_sales = sales_data.groupby('Date')['Units'].sum()

# Fit an ARIMA model to predict future demand
model = ARIMA(daily_sales, order=(5,1,0))  # Example parameters (p, d, q)
arima_model = model.fit()

# Forecast the next 30 days (i.e., for the next month)
forecast = arima_model.forecast(steps=30)
print(forecast)

# Plot the results
plt.plot(daily_sales, label="Actual Sales")
plt.plot(forecast, label="Predicted Sales", color="red")
plt.legend()
plt.show()


In [ ]:
## Using Random Forest Regressor for Machine Learning Prediction:

In [ ]:
# Define features and target variable
features = ['Price', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_7']  # Can include more features
target = 'Units'

# Split the data into training and testing sets
X = sales_data[features]
y = sales_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict future demand
predictions = rf_model.predict(X_test)

# Calculate model accuracy using RMSE or MAE
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE: {rmse}')


In [ ]:
## Step 4: Price Optimization

In [ ]:
from sklearn.linear_model import LinearRegression

# Prepare the data for price elasticity analysis
elasticity_data = sales_data[['Price', 'Units']]

# Add log-transformed columns for regression
elasticity_data['Log_Price'] = np.log(elasticity_data['Price'])
elasticity_data['Log_Units'] = np.log(elasticity_data['Units'])

# Fit a linear regression model to estimate price elasticity
X = elasticity_data[['Log_Price']]
y = elasticity_data['Log_Units']
elasticity_model = LinearRegression()
elasticity_model.fit(X, y)

# The coefficient represents price elasticity
elasticity = elasticity_model.coef_[0]
print(f"Price Elasticity: {elasticity}")


In [ ]:
## Finding Optimal Price:

In [ ]:
# Assuming demand follows a simple linear relationship, calculate the optimal price
# Objective: maximize revenue (price * demand)

optimal_price = - (elasticity_model.intercept_ / (2 * elasticity))
print(f"Optimal Price: {optimal_price}")


In [ ]:
Next Steps:
Evaluate Models: Test the prediction model on unseen data and calculate additional metrics such as Mean Absolute Error (MAE).
Price Optimization: The price elasticity model can be further refined by incorporating additional variables like promotions or competition data.